In [1]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
from azureml.core import Workspace, Model
from azure.storage.blob import BlobServiceClient
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
import joblib, os
from pyspark.sql import SparkSession
from pyspark.ml.pipeline import PipelineModel
from flask import Flask, request, jsonify

In [3]:
# Initialize Spark session
spark = SparkSession.builder.appName('ml_model_deployment').getOrCreate()
spark

24/11/26 00:13:30 WARN Utils: Your hostname, n016064171 resolves to a loopback address: 127.0.0.1; using 10.0.0.4 instead (on interface eth0)
24/11/26 00:13:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/26 00:13:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
model = PipelineModel.load('ml_pipeline')
model

PipelineModel_22a4ea7120e3

In [5]:
ws = Workspace.from_config()
ws

Workspace.create(name='amazon-products-etl-ml-ws', subscription_id='a7e5bb0a-4c2d-4770-8ba2-5adf2a916d3a', resource_group='etl-and-ml-project-ws')

In [6]:
# model = Model.register(workspace=ws,model_name='amazon_products_price_predictor',model_path='amazon_products_price_predictor.pkl/')
# model

In [7]:
# key_vault = ws.get_default_keyvault()
# key_vault

In [8]:
# secret_name = "amazonproducts-sa-access-key"
# secret_value = key_vault.get_secret(secret_name)

In [9]:
# os.environ['STORAGE_ACCOUNT_ACCESS_KEY'] = secret_value

In [10]:
# storage_account_name = 'amazonproductsdl'
# storage_account_access_key = secret_value
# container_name = 'model'
# blob_name = 'amazon_products_price_predictor.pkl'

In [11]:
# # Create BlobServiceClient
# connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_access_key};EndpointSuffix=core.windows.net"
# blob_service_client = BlobServiceClient.from_connection_string(connection_string)
# blob_service_client

In [12]:
# # Get the BlobClient
# lr_model = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
# lr_model

In [13]:
# scaler = blob_service_client.get_blob_client(container=container_name,blob='scaler.pkl')
# scaler

In [14]:
# ml_pipeline = blob_service_client.get_blob_client(container=container_name,blob='ml_pipeline')
# ml_pipeline

In [15]:
# local_pipeline_path = "lr_pipeline"

# with open(local_pipeline_path, "wb") as file:
#     file.write(ml_pipeline.download_blob().readall())

In [16]:
# local_model_path = "model.pkl"
# with open(local_model_path, "wb") as file:
#     file.write(lr_model.download_blob().readall())

In [17]:
# local_scaler_path = "scaler.pkl"
# with open(local_scaler_path, "wb") as file:
#     file.write(scaler.download_blob().readall())

In [18]:
app = Flask(__name__)
app

<Flask '__main__'>

In [19]:
@app.route("/predict",methods=['POST'])
def predict():
    if request.method == 'POST':
        input_data = request.json
        input_df = spark.createDataFrame(input_data['data'])
        predictions = model.transform(input_df)
        prediction_values = predictions.select('prediction').rdd.flatMap(lambda x: x).collect()
        return jsonify({'predictions': prediction_values})

In [22]:

if __name__ == '__main__':
    app.run(host='0.0.0.0',port=5000)

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.0.0.4:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Nov/2024 00:18:14] "POST /predict HTTP/1.1" 200 -             
